In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def relative_error(pred, truth, reduce):
    mask = (truth > 0.1).float()
    abs_diff = torch.abs(pred-truth)*mask
    rel = reduce(abs_diff/truth)
    return rel

def n_rel(pred, truth, percent):
    rel = relative_error(pred, truth, lambda x: x)
    n = torch.sum(rel < percent)
    return n

In [0]:
import nyudepth as nyu
import torch.utils.data as data
import nyutransforms as T
import mynn
import torch.nn.functional as F
import torch

def print_err(net):
    transforms = T.Compose([T.ToTensor()])
    dataset = nyu.NYUDepthV2Dataset('/content/drive/My Drive/zavrsni/data/nyudata', transform=transforms)
    dataloader = data.DataLoader(dataset, batch_size=16, shuffle=False)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net.to(device)
    net.eval()

    n_val_pixels = 0
    n_pixels = 0
    rel = 0
    n = 0

    for i, sample in enumerate(dataloader):
        inputs, depths = sample['image'], sample['depth']
        inputs, depths = inputs.to(device), depths.to(device)
        n_val_pixels += torch.sum(depths > 0.1).item()
        outputs = net(inputs)
        outputs = F.interpolate(outputs, size=(480, 640), mode='bilinear', align_corners=False)
        rel += relative_error(outputs, depths, torch.sum).item()
        n += n_rel(outputs, depths, 0.25).item()

    print('    Prosječna relativna pogreška:', rel/n_val_pixels)
    print('    Postotak piksela u kojima je relativna pogreška dubine manja od 25%:', float(n)/float(n_val_pixels))

In [3]:
import torch
import mynn

net = mynn.create_mynet()
net.load_state_dict(torch.load('/content/drive/My Drive/zavrsni/nyu_netSGD3.pt'))
print('Točnost na trainu za berHu loss:')
print_err(net)




Točnost na trainu za berHu loss:
    Prosječna relativna pogreška: 0.4399972776260586
    Postotak piksela u kojima je relativna pogreška dubine manja od 25%: 0.8602220243091425


In [4]:
import torch
import mynn

net = mynn.create_mynet()
net.load_state_dict(torch.load('/content/drive/My Drive/zavrsni/nyu_netL2.pt'))
print('Točnost na trainu za L2 loss:')
print_err(net)

Točnost na trainu za L2 loss:
    Prosječna relativna pogreška: 0.6413704380398045
    Postotak piksela u kojima je relativna pogreška dubine manja od 25%: 0.7796321532080436
